In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

In [3]:
ds= pd.read_csv('/content/drive/MyDrive/diabetes_data.csv')
ds.head()

,Age,Gender,Polyuria,Polydipsia,Sudden_weight_loss,weakness,Polyphagia,Genital_thrush,visual_blurring,Itching,Irritability,delayed_healing,partial_paresis,muscle_stiffness,Alopecia,Obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive


In [4]:
ds.dtypes

Age                    int64
Gender                object
Polyuria              object
Polydipsia            object
Sudden_weight_loss    object
weakness              object
Polyphagia            object
Genital_thrush        object
visual_blurring       object
Itching               object
Irritability          object
delayed_healing       object
partial_paresis       object
muscle_stiffness      object
Alopecia              object
Obesity               object
class                 object
dtype: object

In [5]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Age                 520 non-null    int64 
 1   Gender              520 non-null    object
 2   Polyuria            520 non-null    object
 3   Polydipsia          520 non-null    object
 4   Sudden_weight_loss  520 non-null    object
 5   weakness            520 non-null    object
 6   Polyphagia          520 non-null    object
 7   Genital_thrush      520 non-null    object
 8   visual_blurring     520 non-null    object
 9   Itching             520 non-null    object
 10  Irritability        520 non-null    object
 11  delayed_healing     520 non-null    object
 12  partial_paresis     520 non-null    object
 13  muscle_stiffness    520 non-null    object
 14  Alopecia            520 non-null    object
 15  Obesity             520 non-null    object
 16  class               520 no

In [6]:
import re
def func(x):
  if x == "Male":
    return "1"
  else:
    return "0" 

ds['Gender']=ds['Gender'].apply(lambda x : func(x))



In [7]:
import re
def func(x):
  if x == "Positive":
    return "1"
  else:
    return "0" 

ds['class']=ds['class'].apply(lambda x : func(x))

In [8]:
import re
def func(x):
  if x == "Yes":
    return "1"
  else:
    return "0" 
ds['Polyuria']=ds['Polyuria'].apply(lambda x : func(x))
ds['Polydipsia']=ds['Polydipsia'].apply(lambda x : func(x))
ds['Sudden_weight_loss']=ds['Sudden_weight_loss'].apply(lambda x : func(x))
ds['weakness']=ds['weakness'].apply(lambda x : func(x))
ds['Polyphagia']=ds['Polyphagia'].apply(lambda x : func(x))
ds['Genital_thrush']=ds['Genital_thrush'].apply(lambda x : func(x))
ds['visual_blurring']=ds['visual_blurring'].apply(lambda x : func(x))
ds['Itching']=ds['Itching'].apply(lambda x : func(x))
ds['Irritability']=ds['Irritability'].apply(lambda x : func(x))
ds['delayed_healing']=ds['delayed_healing'].apply(lambda x : func(x)) 
ds['partial_paresis']=ds['partial_paresis'].apply(lambda x : func(x))
ds['muscle_stiffness']=ds['muscle_stiffness'].apply(lambda x : func(x))  
ds['Alopecia']=ds['Alopecia'].apply(lambda x : func(x))
ds['Obesity']=ds['Obesity'].apply(lambda x : func(x))

In [9]:
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
for column in ds.columns:
    ds[column] = labelencoder.fit_transform(ds[column])

In [10]:
factor_columns=ds[['Age', 'Gender', 'Polyuria', 'Polydipsia', 'Sudden_weight_loss',
       'weakness', 'Polyphagia', 'Genital_thrush', 'visual_blurring',
       'Itching', 'Irritability', 'delayed_healing', 'partial_paresis',
       'muscle_stiffness', 'Alopecia', 'Obesity']]

In [11]:
dia_result=ds['class']
dia_result.head()

0    1
1    1
2    1
3    1
4    1
Name: class, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split( factor_columns, dia_result, test_size=0.3, random_state=5) 
X_train, X_test_validation, y_train, y_test_validation = train_test_split(X_train, y_train, test_size=0.3)

In [16]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [17]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential

model=Sequential()
model.add(Dense(12, input_dim=16,activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(2, activation='sigmoid'))


In [18]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy',f1_m,precision_m,recall_m])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                204       
                                                                 
 dense_1 (Dense)             (None, 15)                195       
                                                                 
 dense_2 (Dense)             (None, 8)                 128       
                                                                 
 dense_3 (Dense)             (None, 10)                90        
                                                                 
 dense_4 (Dense)             (None, 2)                 22        
                                                                 
Total params: 639
Trainable params: 639
Non-trainable params: 0
_________________________________________________________________


In [19]:
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test_validation, y_test_validation))

Epoch 1/50
8/8 [==============================] - 3s 146ms/step - loss: 0.7039 - accuracy: 0.4882 - f1_m: 0.9554 - precision_m: 0.6294 - recall_m: 2.0000 - val_loss: 0.6816 - val_accuracy: 0.6273 - val_f1_m: 0.9174 - val_precision_m: 0.5993 - val_recall_m: 2.0000
Epoch 2/50
8/8 [==============================] - 0s 19ms/step - loss: 0.6756 - accuracy: 0.6299 - f1_m: 0.9551 - precision_m: 0.6302 - recall_m: 2.0000 - val_loss: 0.6629 - val_accuracy: 0.6273 - val_f1_m: 0.9174 - val_precision_m: 0.5993 - val_recall_m: 2.0000
Epoch 3/50
8/8 [==============================] - 0s 25ms/step - loss: 0.6576 - accuracy: 0.6299 - f1_m: 0.9544 - precision_m: 0.6292 - recall_m: 2.0000 - val_loss: 0.6590 - val_accuracy: 0.6273 - val_f1_m: 0.9174 - val_precision_m: 0.5993 - val_recall_m: 2.0000
Epoch 4/50
8/8 [==============================] - 0s 19ms/step - loss: 0.6567 - accuracy: 0.6299 - f1_m: 0.9566 - precision_m: 0.6302 - recall_m: 2.0000 - val_loss: 0.6554 - val_accuracy: 0.6273 - val_f1_m: 0.9

In [20]:
prediction_features = model.predict(X_test)
performance = model.evaluate(X_test, y_test)
print(performance)

5/5 [==============================] - 0s 7ms/step - loss: 0.2208 - accuracy: 0.9295 - f1_m: 0.8097 - precision_m: 0.5751 - recall_m: 1.3800
[0.22079069912433624, 0.9294871687889099, 0.809725284576416, 0.575056254863739, 1.3800396919250488]


In [21]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy " + str(accuracy) + ":\n\ff1_score = " + str(f1_score) +
          "\nPrecision = " + str(precision) + "\nRecall = " + str(recall))

Accuracy 0.9294871687889099:
f1_score = 0.809725284576416
Precision = 0.575056254863739
Recall = 1.3800396919250488
